In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount('/content/gdrive')
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from datetime import datetime

Mounted at /content/gdrive


In [2]:
raw_data=pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Absenteeism-data.csv')
raw_data.head()
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         700 non-null    int64  
 1   Reason for Absence         700 non-null    int64  
 2   Date                       700 non-null    object 
 3   Transportation Expense     700 non-null    int64  
 4   Distance to Work           700 non-null    int64  
 5   Age                        700 non-null    int64  
 6   Daily Work Load Average    700 non-null    float64
 7   Body Mass Index            700 non-null    int64  
 8   Education                  700 non-null    int64  
 9   Children                   700 non-null    int64  
 10  Pets                       700 non-null    int64  
 11  Absenteeism Time in Hours  700 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 65.8+ KB


In [3]:
print(raw_data.columns)
raw_data.describe()

Index(['ID', 'Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index',
       'Education', 'Children', 'Pets', 'Absenteeism Time in Hours'],
      dtype='object')


,ID,Reason for Absence,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,17.951429,19.411429,222.347143,29.892857,36.417143,271.801774,26.737143,1.282857,1.021429,0.687143,6.761429
std,11.028144,8.356292,66.312960,14.804446,6.379083,40.021804,4.254701,0.668090,1.112215,1.166095,12.670082
min,1.000000,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,1.000000,0.000000,0.000000,0.000000
25%,9.000000,13.000000,179.000000,16.000000,31.000000,241.476000,24.000000,1.000000,0.000000,0.000000,2.000000
50%,18.000000,23.000000,225.000000,26.000000,37.000000,264.249000,25.000000,1.000000,1.000000,0.000000,3.000000
75%,28.000000,27.000000,260.000000,50.000000,40.000000,294.217000,31.000000,1.000000,2.000000,1.000000,8.000000
max,36.000000,28.000000,388.000000,52.000000,58.000000,378.884000,38.000000,4.000000,4.000000,8.000000,120.000000


In [4]:
df_drop_id=raw_data.drop(['ID'],axis=1)
df_drop_id.head()

,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [5]:
reason_columns=pd.get_dummies(df_drop_id['Reason for Absence']).drop(0,axis=1)
print(reason_columns)

     1   2   3   4   5   6   7   8   9   10  ...  18  19  21  22  23  24  25  \
0     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
1     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   1   0   0   
3     0   0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   
4     0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   1   0   0   
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
695   0   0   0   0   0   0   0   0   0   1  ...   0   0   0   0   0   0   0   
696   0   0   0   0   0   1   0   0   0   0  ...   0   0   0   0   0   0   0   
697   0   0   0   0   0   0   0   0   0   1  ...   0   0   0   0   0   0   0   
698   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   1   0   0   
699   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   

     26  27  28  
0     1   0   0  
1  

In [6]:
reason_first=reason_columns.iloc[:,:14].max(axis=1)
df_reason_first=pd.DataFrame(reason_first)
reason_second=reason_columns.iloc[:,14:17].max(axis=1)
df_reason_second=pd.DataFrame(reason_second)
reason_third=reason_columns.iloc[:,17:20].max(axis=1)
df_reason_third=pd.DataFrame(reason_third)
reason_fourth=reason_columns.iloc[:,20:].max(axis=1)
df_reason_fourth=pd.DataFrame(reason_fourth)

In [7]:
df_drop_reason=df_drop_id.drop(['Reason for Absence'], axis = 1)
df_reason_categorized=pd.concat([reason_first,reason_second,reason_third,reason_fourth,df_drop_reason], axis=1)
print(df_reason_categorized)

     0  1  2  3        Date  Transportation Expense  Distance to Work  Age  \
0    0  0  0  1  07/07/2015                     289                36   33   
1    0  0  0  0  14/07/2015                     118                13   50   
2    0  0  0  1  15/07/2015                     179                51   38   
3    1  0  0  0  16/07/2015                     279                 5   39   
4    0  0  0  1  23/07/2015                     289                36   33   
..  .. .. .. ..         ...                     ...               ...  ...   
695  1  0  0  0  23/05/2018                     179                22   40   
696  1  0  0  0  23/05/2018                     225                26   28   
697  1  0  0  0  24/05/2018                     330                16   28   
698  0  0  0  1  24/05/2018                     235                16   32   
699  0  0  0  1  31/05/2018                     291                31   40   

     Daily Work Load Average  Body Mass Index  Education  Child

In [8]:
columns = ['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Date', 'Transportation Expense', 'Distance to Work', 'Age',
 'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 'Pets',
 'Absenteeism Time in Hours']
df_reason_categorized.columns = columns
print(df_reason_categorized)

     Reason 1  Reason 2  Reason 3  Reason 4        Date  \
0           0         0         0         1  07/07/2015   
1           0         0         0         0  14/07/2015   
2           0         0         0         1  15/07/2015   
3           1         0         0         0  16/07/2015   
4           0         0         0         1  23/07/2015   
..        ...       ...       ...       ...         ...   
695         1         0         0         0  23/05/2018   
696         1         0         0         0  23/05/2018   
697         1         0         0         0  24/05/2018   
698         0         0         0         1  24/05/2018   
699         0         0         0         1  31/05/2018   

     Transportation Expense  Distance to Work  Age  Daily Work Load Average  \
0                       289                36   33                  239.554   
1                       118                13   50                  239.554   
2                       179                51   38    

In [9]:
df_reason_categorized['Date'] = pd.to_datetime(df_reason_categorized['Date'], format="%d/%m/%Y")
df_reason_categorized['Day of the week'] = df_reason_categorized['Date'].dt.dayofweek
df_reason_categorized['Month'] = df_reason_categorized['Date'].dt.month
df_reason_categorized.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Day of the week,Month
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,1,7
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,1,7
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,2,7
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,3,7
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,3,7


In [10]:
df_date_drop=df_reason_categorized.drop('Date',axis=1)
df_date_drop.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Day of the week,Month
0,0,0,0,1,289,36,33,239.554,30,1,2,1,4,1,7
1,0,0,0,0,118,13,50,239.554,31,1,1,0,0,1,7
2,0,0,0,1,179,51,38,239.554,31,1,0,0,2,2,7
3,1,0,0,0,279,5,39,239.554,24,1,2,0,4,3,7
4,0,0,0,1,289,36,33,239.554,30,1,2,1,2,3,7


In [11]:
edu_list = df_date_drop['Education'].values
new_edu = []
for i in edu_list:
  if i == 1:
    a = 0
    new_edu.append(a)
  else:
    a = 1
    new_edu.append(a)

print(new_edu)

df_date_drop.shape



[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 

(700, 15)

In [12]:
df_edu_drop=df_date_drop.drop('Education', axis = 1)
df_edu_drop['Education'] = new_edu
df_edu_drop


,Reason 1,Reason 2,Reason 3,Reason 4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Children,Pets,Absenteeism Time in Hours,Day of the week,Month,Education
0,0,0,0,1,289,36,33,239.554,30,2,1,4,1,7,0
1,0,0,0,0,118,13,50,239.554,31,1,0,0,1,7,0
2,0,0,0,1,179,51,38,239.554,31,0,0,2,2,7,0
3,1,0,0,0,279,5,39,239.554,24,2,0,4,3,7,0
4,0,0,0,1,289,36,33,239.554,30,2,1,2,3,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,179,22,40,237.656,22,2,0,8,2,5,1
696,1,0,0,0,225,26,28,237.656,24,1,2,3,2,5,0
697,1,0,0,0,330,16,28,237.656,25,0,0,8,3,5,1
698,0,0,0,1,235,16,32,237.656,25,0,0,2,3,5,1


In [13]:
columns_names=['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month', 'Day of the week',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours']
df=df_edu_drop[['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month', 'Day of the week',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours']]
print(df)

     Reason 1  Reason 2  Reason 3  Reason 4  Month  Day of the week  \
0           0         0         0         1      7                1   
1           0         0         0         0      7                1   
2           0         0         0         1      7                2   
3           1         0         0         0      7                3   
4           0         0         0         1      7                3   
..        ...       ...       ...       ...    ...              ...   
695         1         0         0         0      5                2   
696         1         0         0         0      5                2   
697         1         0         0         0      5                3   
698         0         0         0         1      5                3   
699         0         0         0         1      5                3   

     Transportation Expense  Distance to Work  Age  Daily Work Load Average  \
0                       289                36   33                  

In [14]:
path = '/content/gdrive/MyDrive/Colab Notebooks/df_preprocessing.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)